# Reversible Computing
---

In the [previous chapter](./01_01_digital_logic.ipynb), we introduced the concept of Boolean operators such as the binary operations **AND** and **OR**. A critical aspect of these elements is that they are **not** reversible. In other words, we cannot figure out the value of their inputs based solely on the value of the output. 

In case this is not entirely obvious, lets look again at the truth table for the **AND** operation:

| $b$ | $a$ | $b \land a$ |
| :-: | :-: | :-: |
| $0$ | $0$ | $0$ |
| $0$ | $1$ | $0$ |
| $1$ | $0$ | $0$ |
| $1$ | $1$ | $1$ |


Here we can see that if our output is $1$, we can confidently affirm that our inputs are $b = 1$ and $a = 1$. However, if the output is $0$, it is not possible to predict with certainty what the values of $a$ and $b$ are. This uncertainty is equivalent to a loss of information in our system.

[Reversible computing](https://en.wikipedia.org/wiki/Reversible_computing) was born from the idea that the loss of information in classical Boolean circuits is associated with power dissipation [[Landauer61]()]. In an effort to find more power-efficient ways to implement circuits, a model of reversible computation emerged [[Bennett73]()], and with it, a corresponding set of logical gate abstractions to perform these computations [[Toffoli80]()], [[Fredkin81]()].

It was very quickly realized that a model of computation at the quantum-mechanical level can be developed following these same rules [[Feynman85]()], and expanded to include non-classical operations such as superposition and entanglement [[Deutsch85]()], [[Margolus89]()]. Therefore, understanding the basic principles of reversible computing is an important step towards a model of quantum computation.

## 1. Reversible Logic

We begin by analyzing the basic operations we reviewed in the previous chapter in an effort to make them reversible. 
<br></br>

### 1.1 Reversible **NOT**
The first thing to note is that the **NOT** gate ***is*** reversible. This because we can confidently predict the input value directly from its output. If the output is $1$, we know for a fact the input was $0$, and vice versa. Furthermore, we can reverse the computation of a **NOT** gate by applying another **NOT** right after.

Now, because the symbol for the logic **NOT** gate is not symmetric, Feynman suggested changing it to simply using an X over a wire. However, from now on, we will instead use the symbol commonly utilized in quantum computing circuits, which is an X inside a box$^*$:


<img src="images\01_02_01_not_gates.png" align = "center" width="660"/>

For this same reason, we will now refer to the **NOT** gate as the **X** gate.

<span style="font-size: smaller;">$^*$**Aside**: The fact that Feynman used an X to denote the circuit diagram for a reversible **NOT** gate is merely coincidental with the notion of using an X as the symbol for its corresponding quantum gate. The reason why we call the quantum **NOT** an **X** gate has to do with its relationship with the Pauli-X matrix (a topic that we will cover in detail in a later chapter).</span>

### 1.2 Reversible **XOR**
The reversible version of the **XOR** gate is perhaps the most fundamental 2-bit operation in reversible computing and one of the most widely used gates in quantum circuits. We implement this gate by the used of a controlled-X or **CX** gate. A **CX** gate has a control line and a target line. The value of the control line determines if an **X** gate on the target line gets applied or not. If the control line is $0$, the target line is left unchanged; if the control is $1$, the value of the target line is inverted:

<img src="images\01_02_02_cx_gate.png" align = "center" width="210"/>

Following this logic, we can make a truth table for what the outputs $a'$ and $b'$ look like with respect to the inputs $a$ and $b$, and notice that $b' = a \oplus b$:
| $a$ | $b$ | $a'$ | $b'$ |
| :-: | :-: | :-: | :-: |
| $0$ | $0$ | $0$ | $0$ |
| $0$ | $1$ | $0$ | $1$ |
| $1$ | $0$ | $1$ | $1$ |
| $1$ | $1$ | $1$ | $0$ |

**Nice thing is we can make it reversible. Adding another **CX** gate right after, reverses (uncomputes) the operation of the first **CX**. The penalty we had to pay was to double our number of outputs by carrying out one of the input variables (in this case $a$).**


### 1.2 Reversible **COPY**
In the context of classical digital circuits, the **COPY** or **FANOUT** operation is typically envisioned in terms of its physical implementation, which is simply a splitting of wires carrying a common voltage. Clearly, this is not a reversible operation, since we were to However, how would the abstraction of

TALK ABOUT HOW IN QC WE CAN COPY CLASSICAL INFO, BUT NOT GENERAL QUANTUM STATES.


Example of footnote.<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1)

Example of footnote.<a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2)


<a name="cite_note-1"></a>1. [^](#cite_ref-1) footnote 1

<a name="cite_note-2"></a>2. [^](#cite_ref-2) footnote 2